# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

In [5]:
# define indexes for the columns in the event_datafile_new.csv file
IDX_ARTIST = 0
IDX_FIRSTNAME = 1
IDX_GENDER = 2
IDX_ITEMINSESSION = 3
IDX_LASTNAME = 4
IDX_LENGTH = 5
IDX_LEVEL = 6
IDX_LOCATION = 7
IDX_SESSIONID = 8
IDX_SONG = 9
IDX_USERID = 10

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
from cassandra.cluster import Cluster
HOST = '127.0.0.1' # locally installed Apache Cassandra instance

try:
    # establish Cassandra connection
    cluster = Cluster([HOST])
    # establish session to preform queries
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [7]:
# Create a Keyspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    """
)
except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




> **Design Notes.**  
> 1.  As a best practice, create one table for each SQL statement.
> 2.  In terms of table design, put the columns used in the SQL criteria at the beginning and in 
the order referenced in the criteria.

In [9]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query = """
CREATE TABLE IF NOT EXISTS artist_played(
    sessionId       INT
    , itemInSession INT
    , artist        TEXT
    , song          TEXT
    , length        FLOAT        
    , PRIMARY KEY(sessionId, itemInSession)
)
"""
try:
    session.execute(query)
except Exception as e:
    print(e)
    

> For a given session and item in session, identify the artist, song title, and song length.  
Essentially, identify information about the song and artist played at a given moment.
Create table, artist_played, with the relevant columns.  

> Exploring the data reveal that sessionId and itemInSession are tied to userId.  In 
other words, a given user may have one or more session associations (sessionId).  And 
each session may have one or more items (itemInSession) associated with it.  Knowing
this, define a primary key of sessionId and itemInSession.

In [10]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Define INSERT statement
        query = """
        INSERT INTO artist_played(
            sessionId
            , itemInSession
            , artist
            , song
            , length                
        ) VALUES (%s, %s, %s, %s, %s)
        """
        ## Assign column element to each column in the INSERT statement.
        session.execute(query
                        , (int(line[IDX_SESSIONID])
                          , int(line[IDX_ITEMINSESSION])
                          , line[IDX_ARTIST]
                          , line[IDX_SONG]
                          , float(line[IDX_LENGTH]))
                       )


#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
## Verify the data was entered into the table
query = """
SELECT artist
       , song
       , length
FROM   artist_played
WHERE sessionId = 338
AND   itemInSession = 4
"""
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)


Faithless Music Matters (Mark Knight Dub) 495.30731201171875


In [12]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query = """
CREATE TABLE IF NOT EXISTS user_playlist(
    userid          INT
    , sessionId     INT
    , itemInSession INT
    , artist        TEXT
    , song          TEXT
    , firstName     TEXT
    , lastName      TEXT
    , PRIMARY KEY((userid, sessionid), itemInSession)
)
"""
try:
    session.execute(query)
except Exception as e:
    print(e)

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """
        INSERT INTO user_playlist(
            userid
            , sessionId
            , itemInSession
            , artist
            , song
            , firstName
            , lastName
        ) VALUES (%s, %s, %s, %s, %s, %s, %s)
        """
        session.execute(query
                        , (int(line[IDX_USERID])
                          , int(line[IDX_SESSIONID])
                          , int(line[IDX_ITEMINSESSION])
                          , line[IDX_ARTIST]
                          , line[IDX_SONG]
                          , line[IDX_FIRSTNAME]
                          , line[IDX_LASTNAME])
                       )

query = """
SELECT artist
       , song
       , firstName
       , lastName
FROM   user_playlist
WHERE userId = 10
AND   sessionId = 182
"""
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname)    

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [13]:
# verify the songs are ordered correctly by displaying itemInSession column
query = """
SELECT artist
       , song
       , firstName
       , lastName
       , itemInSession
FROM   user_playlist
WHERE userId = 10
AND   sessionId = 182
"""
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname, row.iteminsession)


Down To The Bone Keep On Keepin' On Sylvie Cruz 0
Three Drives Greece 2000 Sylvie Cruz 1
Sebastien Tellier Kilometer Sylvie Cruz 2
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz 3


> For a given user and session, identify the artist, song title, and artist name. 
Create table, user_playlist, with the relevant columns to identify the songs played 
for a particular user.  Since the results must be sorted by itemInSession, we define
this column as a clustering key.  Thus, define a compound partition key composed 
of sessionId and itemInSession with a cluster key column of itemInSession.

> To verify the songs are sorted correctly, display the `itemInSession` column.

In [14]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = """
CREATE TABLE IF NOT EXISTS song_listeners(
    song            TEXT
    , sessionId     INT
    , itemInSession INT
    , firstName     TEXT
    , lastName      TEXT
    , PRIMARY KEY(song, sessionid, itemInSession)
)
"""
try:
    session.execute(query)
except Exception as e:
    print(e)

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """
        INSERT INTO song_listeners(
            song
            , sessionId
            , itemInSession
            , firstName
            , lastName
        ) VALUES (%s, %s, %s, %s, %s)
        """
        session.execute(query
                        , (line[IDX_SONG]
                          , int(line[IDX_SESSIONID])
                          , int(line[IDX_ITEMINSESSION])
                          , line[IDX_FIRSTNAME]
                          , line[IDX_LASTNAME])
                       )

query = """
SELECT firstName
       , lastName
FROM   song_listeners
WHERE song = 'All Hands Against His Own'
"""
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname, row.lastname)


Sara Johnson
Jacqueline Lynch
Tegan Levine


> Identify the users who listened to a particular song.  Create table, song_listeners, 
with names of the users for a particular song.  Since a song could be played by many users
at different points in time (even repeated), define a compound primary key of the columns
song, sessionId, and itemInSession.

### Drop the tables before closing out the sessions

In [15]:
## Drop the table before closing out the sessions
query = "DROP TABLE "
table_list = ['artist_played', 'user_playlist', 'song_listeners']

try:
    for i in table_list:
        rows = session.execute(query + i)
        print("...dropped table " + i)
except Exception as e:
    print(e)


...dropped table artist_played
...dropped table user_playlist
...dropped table song_listeners


### Close the session and cluster connection¶

In [16]:
session.shutdown()
cluster.shutdown()